In [ ]:
import pandas as pd
import numpy as np

In [ ]:
compustat = pd.read_csv("Compustat19612018.csv")
crsp = pd.read_csv("CRSP2007_2018.csv")
linking_table = pd.read_csv("CompustatMergedDatabase.csv")

In [ ]:
# We need link gvkey to compustat if compustat.datadate is between linking_table.LINKDT and linking_table.LINKENDDT
linking_table.head()

In [ ]:
crsp.head()

In [ ]:
# function to change E to today()
from datetime import date
def change_LINKENDDT(x):
    if(x=='E'): x = date.today()
    else : pass
    return x
change_LINKENDDT_vectorize = np.vectorize(change_LINKENDDT)

In [ ]:
linking_table['LINKENDDT_date'] = pd.to_datetime(change_LINKENDDT_vectorize(linking_table['LINKENDDT']),infer_datetime_format=True)
linking_table['LINKDT_date'] = pd.to_datetime(linking_table['LINKDT'],infer_datetime_format=True)
compustat['datadate_date'] = pd.to_datetime(compustat['datadate'],infer_datetime_format=True)

In [ ]:
compustat_linked_join = compustat.set_index('gvkey').join(linking_table.set_index('gvkey'))

In [ ]:
compustat_linked_merge = compustat.merge(linking_table,how='outer',left_on=['gvkey'],right_on=['gvkey'],validate='many_to_many')

In [ ]:
compustat_linked_merge_drop = compustat_linked_merge.drop(compustat_linked_merge[(compustat_linked_merge.datadate_date >= compustat_linked_merge.LINKDT_date) & (compustat_linked_merge.datadate_date <=  compustat_linked_merge.LINKENDDT_date)].index)

In [ ]:
compustat_linked_join.shape

In [ ]:
compustat_linked_merge.shape

In [ ]:
compustat_linked_merge_drop.shape

In [ ]:
compustat.shape

In [ ]:
#df = compustat_linked.drop(compustat_linked[(compustat_linked.datadate >= compustat_linked.LINKDT) & (compustat_linked.datadate <=  compustat_linked.LINKENDDT)].index)

In [ ]:
compustat.groupby(['gvkey','datadate']).size().sort_values(ascending=False).head()